# 모듈 설치 및 설정

In [1]:
import gc

import torch
import optuna
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from os import path as pt
from lib.utils import pickle_it
from sklearn.metrics import f1_score
from torch_geometric.nn import GATv2Conv
from sklearn.preprocessing import StandardScaler

gc.collect()

32

# GAT

- 노드 : 각 시점
- 피처 : 10개 주식 종가 + 예측 기업 거래량 + 예측 기업 감정분석
- GAT의 역할 : 피처들의 관계(회사 간의 관계 등)을 파악해 **시점별** 임베딩 생성;  다른 시점과의 연관성을 반영    
(ex. 1~10일 전과 연결이 되어있는 상태에서, 1일 전 정보는 얼마나 중요하고 10일 전 정보는 얼마나 중요한지 판단)

In [2]:
'''
class GAT(nn.Module):
    def __init__(self, in_features, out_features, heads, dropout, concat=False):
        super(GAT, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.heads = heads
        self.dropout = dropout
        self.concat = concat

        # 선형 변환을 위한 가중치 행렬
        self.lin = nn.Linear(in_features, heads * out_features, bias=False)

        # 어텐션 계수 계산을 위한 가중치
        self.att = nn.Parameter(torch.Tensor(1, heads, out_features))

        # 바이어스
        self.bias = nn.Parameter(torch.Tensor(out_features))

        # LeakyReLU
        self.leakyrelu = nn.LeakyReLU(0.2)

        # 드롭아웃
        self.dropout_layer = nn.Dropout(dropout)

        # 초기화
        self.reset_parameters()

    def reset_parameters(self):
        # 가중치 초기화
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.lin.weight, gain=gain)
        nn.init.xavier_normal_(self.att, gain=gain)
        nn.init.zeros_(self.bias)

    def forward(self, x, edge_index):
        """
        x: 노드 특성 [N, in_features]
        edge_index: 엣지 인덱스 [2, E]
        """
        N = x.size(0)  # 노드 수

        # 1. 선형 변환
        x = self.dropout_layer(x)
        x = self.lin(x)  # [N, heads * out_features]
        x = x.view(N, self.heads, self.out_features)  # [N, heads, out_features]

        # 2. 엣지 리스트로부터 어텐션 계산
        edge_src, edge_dst = edge_index[0], edge_index[1]

        # 3. GATv2 : 선형변환 후 어텐션
        # 각 노드에 어텐션 가중치 적용
        x_att = x * self.att  # [N, heads, out_features]

        # 이웃 노드 쌍의 어텐션 점수 계산
        alpha_src = x_att[edge_src].sum(dim=-1)  # [E, heads]
        alpha_dst = x_att[edge_dst].sum(dim=-1)  # [E, heads]
        alpha = alpha_src + alpha_dst  # [E, heads]
        alpha = self.leakyrelu(alpha)  # [E, heads]

        # 4. 소프트맥스로 정규화
        alpha = self._edge_softmax(alpha, edge_index[1], N)
        alpha = self.dropout_layer(alpha)

        # 5. 메시지 집계
        out = torch.zeros(N, self.heads, self.out_features, device=x.device)

        # 각 엣지에 대해 메시지 전달
        for i in range(edge_index.size(1)):
            src, dst = edge_index[0, i], edge_index[1, i]
            out[dst] += alpha[i].unsqueeze(-1) * x[src]

        # 6. 최종 출력 형태 결정
        if self.concat:
            out = out.view(N, self.heads * self.out_features)
        else:
            out = out.mean(dim=1)  # 헤드 간 평균

        # 7. 바이어스 추가
        out = out + self.bias

        return out

    def _edge_softmax(self, alpha, target_nodes, num_nodes):

        norm_alpha = torch.zeros_like(alpha)


        for i in range(num_nodes):
            # 현재 노드로 향하는 엣지 마스크
            mask = (target_nodes == i)
            if mask.sum() > 0:
                # 해당 노드로 향하는 엣지에 대해 소프트맥스 적용
                norm_alpha[mask] = F.softmax(alpha[mask], dim=0)

        return norm_alpha
 '''

'\nclass GAT(nn.Module):\n    def __init__(self, in_features, out_features, heads, dropout, concat=False):\n        super(GAT, self).__init__()\n\n        self.in_features = in_features\n        self.out_features = out_features\n        self.heads = heads\n        self.dropout = dropout\n        self.concat = concat\n\n        # 선형 변환을 위한 가중치 행렬\n        self.lin = nn.Linear(in_features, heads * out_features, bias=False)\n\n        # 어텐션 계수 계산을 위한 가중치\n        self.att = nn.Parameter(torch.Tensor(1, heads, out_features))\n\n        # 바이어스\n        self.bias = nn.Parameter(torch.Tensor(out_features))\n\n        # LeakyReLU\n        self.leakyrelu = nn.LeakyReLU(0.2)\n\n        # 드롭아웃\n        self.dropout_layer = nn.Dropout(dropout)\n\n        # 초기화\n        self.reset_parameters()\n\n    def reset_parameters(self):\n        # 가중치 초기화\n        gain = nn.init.calculate_gain(\'relu\')\n        nn.init.xavier_normal_(self.lin.weight, gain=gain)\n        nn.init.xavier_normal_(self.att, gai

In [3]:
# GAT 레이어 정의

class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, heads, dropout):
        super().__init__()
        self.gat = GATv2Conv(in_features, out_features, heads=heads, dropout=dropout, concat=False)  # PyG 사용
        # self.gat = GAT(in_features, out_features, heads=heads, dropout=dropout, concat=False)            # 상단 구현 코드 (느리고, 느려서 optuna 중간에 끊고 첫번째걸로 해봤는데 ----로 예측됨;;;;)

    def forward(self, x, edge_index):
        return self.gat(x, edge_index)

# TCN

In [4]:
# --- 유틸 함수 ---
def get_conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias):
    return nn.Conv1d(in_channels=in_channels, out_channels=out_channels,
                     kernel_size=kernel_size, stride=stride,
                     padding=padding, dilation=dilation,
                     groups=groups, bias=bias)


def get_bn(channels):
    return nn.BatchNorm1d(channels)


def conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups, dilation=1, bias=False):
    if padding is None:
        padding = kernel_size // 2
    result = nn.Sequential()
    result.add_module('conv',
                      get_conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias))
    result.add_module('bn', get_bn(out_channels))
    return result


# --- RevIN ---
class RevIN(nn.Module):
    def __init__(self, num_features: int, eps=1e-5, affine=True, subtract_last=False):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.subtract_last = subtract_last
        if self.affine:
            self._init_params()

    def _init_params(self):
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def forward(self, x, mode: str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        return x

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim - 1))
        if self.subtract_last:
            self.last = x[:, -1:, :].unsqueeze(1)
        else:
            self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        x = (x - self.mean) / self.stdev
        if self.affine:
            x = x * self.affine_weight[None, None, :] + self.affine_bias[None, None, :]
        return x

    def _denormalize(self, x):
        if self.affine:
            x = (x - self.affine_bias[None, None, :]) / self.affine_weight[None, None, :]
        x = x * self.stdev + self.mean
        return x


# --- 시계열 분해 ---
class moving_avg(nn.Module):
    def __init__(self, kernel_size, stride):
        super().__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        return x.permute(0, 2, 1)


class series_decomp(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        return x - moving_mean, moving_mean


# --- 커스텀 커널 ---
class ReparamLargeKernelConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, small_kernel, small_kernel_merged=False):
        super().__init__()
        self.kernel_size = kernel_size
        self.small_kernel = small_kernel
        padding = kernel_size // 2
        if small_kernel_merged:
            self.lkb_reparam = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, groups=groups,
                                         bias=True)
        else:
            self.lkb_origin = conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups)
            if small_kernel is not None:
                self.small_conv = conv_bn(in_channels, out_channels, small_kernel, stride, small_kernel // 2, groups)

    def forward(self, x):
        if hasattr(self, 'lkb_reparam'):
            return self.lkb_reparam(x)
        out = self.lkb_origin(x)
        if hasattr(self, 'small_conv'):
            out += self.small_conv(x)
        return out


# --- 출력층 ---
class Flatten_Head(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.linear = nn.Linear(d_model, 1)

    def forward(self, x):  # x: [B, C, T]
        x = x.permute(0, 2, 1)  # → [B, T, C]
        x = self.linear(x)  # → [B, T, 1]
        return x.squeeze(-1)  # → [B, T]

In [5]:
# --- ModernTCN 모델 ---
class ModernTCN(nn.Module):
    def __init__(self, configs):
        super().__init__()
        self.revin = RevIN(configs.enc_in, affine=configs.affine) if configs.revin else None
        self.decomp = series_decomp(configs.kernel_size) if configs.decomposition else None

        self.conv_layers = nn.ModuleList()
        self.norm_layers = nn.ModuleList()

        c_in = configs.enc_in
        for i in range(len(configs.dims)):
            conv = ReparamLargeKernelConv(c_in, configs.dims[i],
                                          kernel_size=configs.large_size[i],
                                          stride=1,
                                          groups=1,
                                          small_kernel=configs.small_size[i],
                                          small_kernel_merged=configs.small_kernel_merged)
            self.conv_layers.append(conv)
            self.norm_layers.append(nn.BatchNorm1d(configs.dims[i]))
            c_in = configs.dims[i]

        self.head = Flatten_Head(configs.dims[-1])

    def forward(self, x):  # x: [B, T, C]
        if self.revin:
            x = self.revin(x, 'norm')
        if self.decomp:
            x, _ = self.decomp(x)
        x = x.permute(0, 2, 1)  # [B, C, T]
        for conv, norm in zip(self.conv_layers, self.norm_layers):
            x = conv(x)
            x = norm(x)
            x = F.relu(x)
        out = self.head(x)  # [B, T]
        return out

In [6]:
# --- Config 클래스 ---
class Configs:
    def __init__(self, enc_in):
        self.enc_in = enc_in
        self.dims = [8, 16, 32]
        self.large_size = [5, 5, 3]
        self.small_size = [5, 3, 3]
        self.small_kernel_merged = False
        self.dropout = 0.1
        self.head_dropout = 0.2
        self.revin = True
        self.affine = True
        self.decomposition = True
        self.kernel_size = 25

# GAT-TCN

In [7]:
class GATCNModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.gat = GATLayer(node_features.shape[1], config.gat_out_features,
                            config.gat_heads, config.gat_dropout)  # GAT
        self.tcn = ModernTCN(config)  # TCN

    def forward(self, x, edge_index):
        embeddings = self.gat(x, edge_index)  # GAT를 통한 임베딩 생성
        tcn_input = embeddings.unsqueeze(0)  # TCN 입력 형태로 변환
        output = self.tcn(tcn_input)  # TCN으로 예측
        return output

In [8]:

# GAT-TCN 모델의 최적 파라미터 탐색

def train_model(model, node_features, edge_index, X_train, y_train, X_val, y_val, epochs=30, lr=1e-3, pos_weight=None):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    #criterion = nn.BCEWithLogitsLoss()
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight) if pos_weight is not None else nn.BCEWithLogitsLoss()
    train_losses, val_losses, val_accs = [], [], []

    for epoch in range(epochs):
        # 1.학습 단계
        model.train()
        optimizer.zero_grad()
        train_output = model(node_features, edge_index).squeeze(0)[:len(y_train)]  # 통합 모델에 데이터를 적용한 결과
        loss = criterion(train_output, y_train.float())
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        # 2. 검증 단계
        model.eval()
        with torch.no_grad():
            val_output = model(node_features, edge_index).squeeze(0)[-len(y_val):]
            val_loss = criterion(val_output, y_val.float()).item()
            pred = (torch.sigmoid(val_output) > 0.5).int()
            acc = (pred == y_val).float().mean().item()

        val_losses.append(val_loss)
        val_accs.append(acc)

        print(f"[{epoch + 1}/{epochs}] Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Val Acc: {acc:.4f}")

    return train_losses, val_losses, val_accs


In [9]:
def objective(trial):
    # GAT
    gat_out_features = trial.suggest_categorical("gat_out_features", [4, 8, 12])  # 기존 features 수보다는 적은 것이 적합
    gat_heads = trial.suggest_categorical("gat_heads", [1, 2, 4, 8])
    gat_dropout = trial.suggest_float("gat_dropout", 0.0, 0.3)

    # TCN
    dims = [
        trial.suggest_categorical("dim1", [8, 16, 32, 64]),
        trial.suggest_categorical("dim2", [16, 32, 64, 128]),
        trial.suggest_categorical("dim3", [32, 64, 128, 256])
    ]
    large_size = [
        trial.suggest_categorical("k1", [3, 5, 7, 9, 11]),
        trial.suggest_categorical("k2", [3, 5, 7, 9]),
        trial.suggest_categorical("k3", [3, 5, 7])
    ]
    small_size = [
        trial.suggest_categorical("s1", [1, 3, 5]),
        trial.suggest_categorical("s2", [1, 3]),
        trial.suggest_categorical("s3", [1, 3])
    ]
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    head_dropout = trial.suggest_float("head_dropout", 0.0, 0.3)
    kernel_size = trial.suggest_categorical("kernel_size", [5, 11, 15, 25, 31])
    decomposition = trial.suggest_categorical("decomposition", [True, False])
    revin = trial.suggest_categorical("revin", [True, False])
    affine = trial.suggest_categorical("affine", [True, False])

    # 통합
    class TrialConfig:
        def __init__(self):
            self.gat_out_features = gat_out_features
            self.gat_heads = gat_heads
            self.gat_dropout = gat_dropout

            self.enc_in = gat_out_features
            self.dims = dims
            self.large_size = large_size
            self.small_size = small_size
            self.small_kernel_merged = False
            self.dropout = dropout
            self.head_dropout = head_dropout
            self.revin = revin
            self.affine = affine
            self.decomposition = decomposition
            self.kernel_size = kernel_size

    model = GATCNModel(TrialConfig())

    # Accuracy 기준 최적화 #####
    _, _, val_accs = train_model(model, node_features, edge_index, X_train, y_train, X_val, y_val, epochs=50)
    return max(val_accs)
    ############################

    # F1 Score 기준 최적화 #####################
    # pos_weight 계산 (불균형 데이터 보정)
    '''
    pos_weight = torch.tensor([(y_train == 0).sum() / (y_train == 1).sum()]).to(y_train.device)

    train_model(model, node_features, edge_index, X_train, y_train, X_val, y_val, epochs=15, pos_weight=pos_weight)

    model.eval()
    with torch.no_grad():
        full_pred = model(node_features, edge_index).squeeze(0)
        pred = full_pred[-len(y_val):]
        probs = torch.sigmoid(pred).cpu().numpy()
        preds = (probs > 0.5).astype(int)

    y_true = y_val.cpu().numpy()
    return f1_score(y_true, preds)
    '''
    ##########################################

# 시각화 코드

In [10]:
def visualize_training(company_name, train_losses, val_losses, val_accs):
    plt.figure(figsize=(12, 2), dpi=400)
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.legend()
    plt.title("Loss over Epochs")

    plt.subplot(1, 2, 2)
    plt.plot(val_accs, label='Val Accuracy')
    plt.legend()
    plt.title("Validation Accuracy")
    plt.savefig(f'data/images/{company_name}_loss.png')
    plt.close()

from matplotlib.patches import Patch

def visualize_match_only(company_name, pred_probs, true_labels, threshold=0.5):
    # 이진 예측
    pred_labels = (pred_probs >= threshold).astype(int)

    # 정답과 예측이 일치하면 1, 다르면 0
    match = (pred_labels == true_labels).astype(int)

    # 시각화
    plt.figure(figsize=(12, 2), dpi=400)
    bar_heights = np.ones_like(match)
    bar_colors = ['green' if m else 'red' for m in match]
    plt.bar(np.arange(len(match)), bar_heights,
            color=bar_colors,
            width=1.0)

    plt.title(f"{company_name} - Prediction Match")
    plt.ylabel('Match')
    plt.xlabel('Time')
    plt.yticks([0, 1], ['Wrong', 'Correct'])

    # ✅ 범례 추가
    legend_elements = [
        Patch(facecolor='green', label='Correct'),
        Patch(facecolor='red', label='Wrong')
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.tight_layout()
    plt.savefig(f'data/images/{company_name}_match_only.png')
    plt.close()


def visualize_cumulative_return(pred_probs, true_labels, prices):
    signal = (pred_probs > 0.5).astype(int)
    returns = (prices[1:] / prices[:-1]) - 1
    strategy_returns = returns * signal[:-1]

    cumulative = (strategy_returns + 1).cumprod()
    market = (returns + 1).cumprod()

    plt.plot(cumulative, label='Strategy')
    plt.plot(market, label='Market (buy & hold)')
    plt.legend();
    plt.title("Cumulative Return")
    plt.show()

# GAT-TCN 적용

In [11]:
# seed 설정
import random


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)


set_seed(42)


1. 데이터셋 불러오기 및 라벨 생성

In [12]:
df_ = pd.read_csv("./data/daily_all.csv")

In [13]:
# company_name =['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN'][0]

In [17]:
for company_name in ['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN']:
    # company_name = "AMZN"  # 예측할 회사 선택
    data = df_[
        [f'prccd_{company}' for company in
         ['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN']] +
        [f'cshtrd_{company_name}', f'sent_{company_name}', 'datadate']].copy()
    data.set_index('datadate', inplace=True)
    data.fillna(0, inplace=True)  # 감정분석 결측값을 0으로
    data.iloc[:, :10] = data.iloc[:, :10].pct_change()
            # returns = (close_prices[1:] / close_prices[:-1]) - 1
        # labels = np.where(returns > 0.003, 1, 0)
    data = data.dropna()
    data_values = data.values
    scaler = StandardScaler()
    data_s = scaler.fit_transform(data_values[:, :-1])
    df_preprocessed = np.hstack([data_s, data_values[:, -1].reshape(-1, 1)])
    node_features = df_preprocessed
    n_nodes = node_features.shape[0]  # 노드 수 (==날짜 수)

    # 그래프 형태로 변환 : 엣지 생성 (시점 간 연결)
    edge_list = []
    for i in range(n_nodes):
        for j in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:  # 이후 시점들에 단방향 연결; (휴장 같은 것은 생각하지 않음.... 시점 기준)
            if i + j < n_nodes:
                edge_list.append([i, i + j])
    edge_index = torch.tensor(edge_list).t()


    # edge_list = []
    # for i in range(n_nodes):
    #     for j in range(1, 11):  # 과거 1~10시점
    #         if i - j >= 0:
    #             edge_list.append([i, i - j])  # 현재 → 과거 방향
    # edge_index = torch.tensor(edge_list).t()

    # 라벨 생성
    close_prices = data[f'prccd_{company_name}'].values
    # returns = (close_prices[1:] / close_prices[:-1]) - 1
    labels = np.where(close_prices > 0.003, 1, 0)  # 0.3% 초과만 1로
    labels = torch.tensor(labels, dtype=torch.float32)
    node_features = torch.tensor(node_features, dtype=torch.float)

    split = int(len(node_features) * 0.8)
    # print(split)
    # print(len(labels))

    # print(edge_index)
    # train_edge_index = edge_index[:split]
    # val_edge_index = edge_index[split:]

    train_node_features = node_features[:split]
    val_node_features = node_features[split:]

    train_labels = labels[:split]
    val_labels = labels[split:]
    X = train_node_features[:-1]
    y = train_labels[1:]
    split = int(len(X) * 0.8)


    X_train = X[:split]
    X_val   = X[split:]

    y_train = y[:split]
    y_val   = y[split:]
    # Optuna 튜닝 실행
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=1)
    # 최적 하이퍼파라미터 출력
    print("✅ Best Trial:")
    print(study.best_trial.params)
    best_params = study.best_trial.params


    # BestConfig를 이용해 모델 설정 후 학습 : optuna를 통해 정해진 최적 하이퍼파라미터

    class BestConfig:
        def __init__(self):
            # GAT 설정
            self.gat_out_features = best_params['gat_out_features']
            self.gat_heads = best_params['gat_heads']
            self.gat_dropout = best_params['gat_dropout']

            # TCN 설정
            self.enc_in = best_params['gat_out_features']  # GAT 출력 = TCN 입력
            self.dims = [best_params['dim1'], best_params['dim2'], best_params['dim3']]
            self.large_size = [best_params['k1'], best_params['k2'], best_params['k3']]
            self.small_size = [best_params['s1'], best_params['s2'], best_params['s3']]
            self.small_kernel_merged = False
            self.dropout = best_params['dropout']
            self.head_dropout = best_params['head_dropout']
            self.revin = best_params['revin']
            self.affine = best_params['affine']
            self.decomposition = best_params['decomposition']
            self.kernel_size = best_params['kernel_size']


    model = GATCNModel(BestConfig())
    train_losses, val_losses, val_accs = train_model(model, train_node_features, edge_index, X_train, y_train, X_val, y_val,
                                                     epochs=75, lr=1e-3)

    with torch.no_grad():
        pred_logits = model(val_node_features, edge_index).squeeze(0)  # [1, T] → [T]
        pred_probs = torch.sigmoid(pred_logits).cpu().numpy()
        pred_labels = (pred_probs > 0.5).astype(int)

    print(company_name)
    visualize_training(company_name, train_losses, val_losses, val_accs)
    visualize_match_only(company_name, pred_probs, val_labels.cpu().numpy())

    df = pd.DataFrame(np.vstack([val_labels.cpu().numpy(), pred_labels])).T
    df.columns = ['y_true', 'y_pred']  # 열 이름 지정

    f1 = f1_score(df['y_true'], df['y_pred'], average='macro')  # or 'micro', 'weighted'
    print(f"F1 Score: {f1:.4f}")
    df.to_csv(f'data/{company_name}.csv')
    pickle_it(model.to('cpu').state_dict(), pt.join('general_results', f'weights_{company_name}.torch'))

[I 2025-05-27 16:24:12,977] A new study created in memory with name: no-name-2095b097-4a35-4103-a142-ea7b8780b8e3


[1/50] Train Loss: 0.7165, Val Loss: 0.7079, Val Acc: 0.4548
[2/50] Train Loss: 0.6933, Val Loss: 0.7073, Val Acc: 0.4548
[3/50] Train Loss: 0.6821, Val Loss: 0.7055, Val Acc: 0.4548
[4/50] Train Loss: 0.6719, Val Loss: 0.7039, Val Acc: 0.4548
[5/50] Train Loss: 0.6599, Val Loss: 0.7034, Val Acc: 0.4525
[6/50] Train Loss: 0.6504, Val Loss: 0.7035, Val Acc: 0.4638
[7/50] Train Loss: 0.6530, Val Loss: 0.7036, Val Acc: 0.4525
[8/50] Train Loss: 0.6464, Val Loss: 0.7039, Val Acc: 0.4661
[9/50] Train Loss: 0.6470, Val Loss: 0.7042, Val Acc: 0.4751
[10/50] Train Loss: 0.6338, Val Loss: 0.7047, Val Acc: 0.4683
[11/50] Train Loss: 0.6278, Val Loss: 0.7054, Val Acc: 0.4729
[12/50] Train Loss: 0.6210, Val Loss: 0.7066, Val Acc: 0.4683
[13/50] Train Loss: 0.6262, Val Loss: 0.7085, Val Acc: 0.4706
[14/50] Train Loss: 0.6279, Val Loss: 0.7110, Val Acc: 0.4638
[15/50] Train Loss: 0.6222, Val Loss: 0.7138, Val Acc: 0.4683
[16/50] Train Loss: 0.6119, Val Loss: 0.7166, Val Acc: 0.4683
[17/50] Train Los

[I 2025-05-27 16:24:53,352] Trial 0 finished with value: 0.4841628968715668 and parameters: {'gat_out_features': 4, 'gat_heads': 2, 'gat_dropout': 0.18119751269098475, 'dim1': 32, 'dim2': 128, 'dim3': 32, 'k1': 9, 'k2': 5, 'k3': 5, 's1': 5, 's2': 3, 's3': 3, 'dropout': 0.20061806721053804, 'head_dropout': 0.0965352363231529, 'kernel_size': 11, 'decomposition': True, 'revin': True, 'affine': False}. Best is trial 0 with value: 0.4841628968715668.


[49/50] Train Loss: 0.5504, Val Loss: 0.8560, Val Acc: 0.4774
[50/50] Train Loss: 0.5273, Val Loss: 0.8547, Val Acc: 0.4706
✅ Best Trial:
{'gat_out_features': 4, 'gat_heads': 2, 'gat_dropout': 0.18119751269098475, 'dim1': 32, 'dim2': 128, 'dim3': 32, 'k1': 9, 'k2': 5, 'k3': 5, 's1': 5, 's2': 3, 's3': 3, 'dropout': 0.20061806721053804, 'head_dropout': 0.0965352363231529, 'kernel_size': 11, 'decomposition': True, 'revin': True, 'affine': False}


IndexError: index out of range in self

3. 모델 생성 및 학습

In [ ]:
# BestConfig를 이용해 모델 설정 후 학습 :  여러 시행, 다양한 회사에서의 bestconfig 정보를 종합해 최적 파라미터 직접 설정 (아래는 예시)

'''
class BestConfig:
    def __init__(self):
        # GAT 설정
        self.gat_out_features = 8
        self.gat_heads = 4
        self.gat_dropout = 0.2

        # TCN 설정
        self.enc_in = 8
        self.dims = [8,16,32]
        self.large_size = [7,5,7]
        self.small_size = [5,1,1]
        self.small_kernel_merged = False
        self.dropout = 0.2
        self.head_dropout = 0.25
        self.revin = True
        self.affine = False
        self.decomposition = False
        self.kernel_size = 31
'''

4. 예측 및 시각화 (best_params 기준으로 설정한 값)

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix
#
# print(classification_report(y_val.cpu(), pred_labels))
# print(confusion_matrix(y_val.cpu(), pred_labels))

In [ ]:


"""
# 1. 실제 라벨
true_labels = y_val.cpu().numpy()

# 2. 다양한 threshold에 대해 f1-score 측정
precisions, recalls, thresholds = precision_recall_curve(true_labels, pred_probs)

f1s = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)  # f1-score 계산
best_idx = np.argmax(f1s)
best_threshold = thresholds[best_idx]

print(f"✅ Best threshold by F1-score: {best_threshold:.4f}, F1: {f1s[best_idx]:.4f}")

# 3. 최적 threshold로 예측 라벨 생성
pred_labels = (pred_probs > best_threshold).astype(int)
# 6. 누적 수익률 (선택)
# future_prices = close_prices[split+1:]  # 실제 수익률 계산용
# visualize_cumulative_return(pred_probs, y_val.cpu().numpy(), future_prices)
"""


In [ ]:
weight_df = pd.DataFrame(index=df_[split+2:]['datadate'])
for company_name in ['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN']:
    company_df = pd.read_csv(f'data/{company_name}.csv', index_col=0)['y_pred']
    company_df.name = company_name
    company_df.index = weight_df.index
    weight_df = pd.concat([weight_df, company_df], axis=1)
    row_sum = (weight_df == 1).sum(axis=1)
    row_sum.replace(0, 0.1, inplace=True)
    weight_df_1=weight_df.div(row_sum, axis=0)
    weight_df_1.to_csv(f'data/GAT_TCN_weight.csv')

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics import f1_score
# from sklearn.preprocessing import StandardScaler
# from lightgbm import LGBMClassifier
#
#
# def train_lgbm_model(X_train, y_train, X_val):
#     model = LGBMClassifier(
#         n_estimators=100,
#         learning_rate=0.1,             # 너무 작으면 과적합/학습 지연
#         verbosity=-1
#     )
#     model.fit(X_train, y_train)
#
#     pred_probs = model.predict_proba(X_val)[:, 1]
#     pred_labels = (pred_probs > 0.5).astype(int)
#
#     return model, pred_probs, pred_labels
#
# # 가상의 node_features, labels, df_ 등이 정의되어 있어야 합니다.
# # 위의 리팩토링은 함수 형태만 준비되어 있고, 본문 루프는 사용자가 가진 데이터 프레임 `df_`가 있어야 실행 가능
#
# def run_lgbm_pipeline(df_):
#     results = []
#     for company_name in ['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN']:
#         data = df_[
#             [f'prccd_{company}' for company in
#              ['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN']] +
#             [f'cshtrd_{company_name}', f'sent_{company_name}', 'datadate']].copy()
#
#         data.set_index('datadate', inplace=True)
#         data.fillna(0, inplace=True)
#         data.iloc[:, :10] = data.iloc[:, :10].pct_change()
#         data = data.dropna()
#         close_prices = data[f'prccd_{company_name}'].values
#
#         # ✅ 라벨: t+1 수익률 기준
#         # returns = (close_prices[1:] / close_prices[:-1]) - 1
#         labels = np.where(close_prices > 0.003, 1, 0)
#
#         # ✅ feature는 시점 t까지 (맨 마지막 row 제거)
#         data_values = data.values
#         scaler = StandardScaler()
#         data_s = scaler.fit_transform(data_values[:, :-1])
#         node_features = np.hstack([data_s, data_values[:, -1].reshape(-1, 1)])
#
#
#         # ✅ train/val split
#         X = node_features[:-1]
#         y = labels[1:]
#         split = int(len(X) * 0.8)
#
#         X_train = X[:split]
#         X_val   = X[split:]
#
#         y_train = y[:split]
#         y_val   = y[split:]
#
#         # ✅ 모델 학습 및 예측
#         model, pred_probs, pred_labels = train_lgbm_model(X_train, y_train, X_val)
#
#         print(company_name)
#         f1 = f1_score(y_val, pred_labels, average='macro')
#         print(f"F1 Score: {f1:.4f}")
#
#         df_result = pd.DataFrame({'y_true': y_val, 'y_pred': pred_labels})
#         df_result.to_csv(f'data/{company_name}_lgbm.csv', index=False)
#
#         results.append({
#             "company": company_name,
#             "f1_score": f1,
#         })
#
#     return results

In [ ]:
result=run_lgbm_pipeline(df_)
result

In [ ]:
weight_df = pd.DataFrame(index=df_[split+2:]['datadate'])
for company_name in ['TSLA', 'NVDA', 'MSFT', 'GOOG', 'AAPL', 'DIS', 'XOM', 'CRM', 'INTC', 'AMZN']:
    company_df = pd.read_csv(f'data/{company_name}_lgbm.csv', index_col=0)['y_pred']
    company_df.name = company_name
    company_df.index = weight_df.index
    weight_df = pd.concat([weight_df, company_df], axis=1)
    row_sum = (weight_df == 1).sum(axis=1)
    row_sum.replace(0, 0.1, inplace=True)
    weight_df_1=weight_df.div(row_sum, axis=0)
    weight_df_1.to_csv(f'data/LGBM_weight.csv')